I have a shapefile with point geometry for all snotel stations in US (about 1k stations). My goal is to get the the snow depth for each of the station for a particular date.

In [75]:
import pandas as pd
import geopandas as gpd
from datetime import datetime
from metloom.pointdata import SnotelPointData
from metloom.pointdata import CDECPointData
from metloom.variables import CdecStationVariables
from metloom.variables import SnotelVariables
import numpy as np


In [114]:
#define the path to the region shapefile and read
idaho = "/home/naheemadebisi/PhD/snow-analytics/Idaho_region.gpkg"
obj = gpd.read_file(idaho)
#specify the variable of interest
snotel_vrs = [SnotelVariables.SNOWDEPTH]
#get snotel points objects
snotel_points = SnotelPointData.points_from_geometry(obj, snotel_vrs)

In [117]:
#create dataframe for the snotel_points
snt_pts_df = snotel_points.to_dataframe()
#create an column for snow depth and fill values with nan
snt_pts_df['snowdepth [inch]'] = np.nan
#set index to the id
snt_pts_df.set_index('id', inplace=True)
snt_pts_df

,name,datasource,geometry,snowdepth [inch]
id,,,,
471:ID:SNTL,Emigrant Summit,NRCS,POINT Z (-111.56085 42.36055 7390.00000),NaN
774:ID:SNTL,South Mtn.,NRCS,POINT Z (-116.90037 42.76478 6500.00000),NaN
319:ID:SNTL,Bear Basin,NRCS,POINT Z (-116.14293 44.95222 5350.00000),NaN
484:ID:SNTL,Franklin Basin,NRCS,POINT Z (-111.60120 42.05050 8140.00000),NaN
653:OR:SNTL,Mt. Howard,NRCS,POINT Z (-117.17373 45.26522 7910.00000),NaN
...,...,...,...,...
664:MT:SNTL,Noisy Basin,NRCS,POINT Z (-113.94637 48.15678 6040.00000),NaN
535:ID:SNTL,Humboldt Gulch,NRCS,POINT Z (-115.77643 47.53178 4250.00000),NaN
690:MT:SNTL,Pickfoot Creek,NRCS,POINT Z (-111.26832 46.57978 6650.00000),NaN


In [125]:
#create a date to get the snowdepth for
start_date = datetime(2020, 2, 9)
end_date = datetime(2020, 2, 9)
#iterate over the values of each points of the points objects to get the daily snow depth value
for pnt in snotel_points:
    df = pnt.get_daily_data(start_date, end_date, [pnt.ALLOWED_VARIABLES.SNOWDEPTH])
    #proceed if we succefully get the snowdepth info into a GeoDataframe
    if isinstance(df, gpd.GeoDataFrame):
        df = df.reset_index()[['site', 'SNOWDEPTH']]
        df.set_index('site', inplace=True)
        snt_pts_df.loc[df.index, 'snowdepth [inch]'] = df['SNOWDEPTH']
    else:
        continue


No SNOWDEPTH found for Couch Summit
No SNOWDEPTH found for Chicago Ridge
No SNOWDEPTH found for Pebble Creek
No SNOWDEPTH found for Slagamelt Lakes
No SNOWDEPTH found for Fish Ck
No SNOWDEPTH found for Schweitzer Saddle


In [130]:
snt_pts_df['snowdepth (m)'] = snt_pts_df['snowdepth [inch]'] * 0.0254


In [131]:
snt_pts_df

,name,datasource,geometry,snowdepth [inch],snowdepth (m)
id,,,,,
471:ID:SNTL,Emigrant Summit,NRCS,POINT Z (-111.56085 42.36055 7390.00000),55.0,1.3970
774:ID:SNTL,South Mtn.,NRCS,POINT Z (-116.90037 42.76478 6500.00000),38.0,0.9652
319:ID:SNTL,Bear Basin,NRCS,POINT Z (-116.14293 44.95222 5350.00000),61.0,1.5494
484:ID:SNTL,Franklin Basin,NRCS,POINT Z (-111.60120 42.05050 8140.00000),66.0,1.6764
653:OR:SNTL,Mt. Howard,NRCS,POINT Z (-117.17373 45.26522 7910.00000),37.0,0.9398
...,...,...,...,...,...
664:MT:SNTL,Noisy Basin,NRCS,POINT Z (-113.94637 48.15678 6040.00000),110.0,2.7940
535:ID:SNTL,Humboldt Gulch,NRCS,POINT Z (-115.77643 47.53178 4250.00000),51.0,1.2954
690:MT:SNTL,Pickfoot Creek,NRCS,POINT Z (-111.26832 46.57978 6650.00000),30.0,0.7620


In [132]:
snt_pts_df.to_file('id_snotelsd_20200209.shp', driver='ESRI Shapefile')

/tmp/ipykernel_295804/14667653.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  snt_pts_df.to_file('id_snotelsd_20200209.shp', driver='ESRI Shapefile')
